In [6]:
# A function that may be called by an optimisation code: 
# pyGMOSS_Func_ComputeVariance_v0(x, flag_param, freq_array, Tb_array)
#
# This function receives a set of variable and static parameters
#        that determine the phyiscal spectrum at each sky pixel.
# And arrays of frequency and brightness.
# Returns a variance measure of deviation between computed spectral values and data
#
# The call to scipy.optimize.minimize would contain
# Initial guess for the variable parameters
# x is a 1-D array with shape (n=7) containing, in this order, the variable parameters:
    # norm_param
    # alpha1_param
    # dalpha_param
    # nubrk_param
    # Tx_param
    # Te_param
    # nut_param
#
# args in the call to the optimisation would be passed to this function, and
# should be a tuple of the fixed parameter flag and arrays containing frequencies and brightness temperatures.
    #
    # flag_param: 0 where alpha2 steeper than alpha1 (convex spectrum)
    #             1 where data requires alpha2 flatter than alpha1 (concave spectrum)
    #             alpha1, alpha2 are defined with convention: T(\nu) proportional to \nu^(-alpha)
    #             alpha2 is the spectral index at higher frequencies, alpha1 at lower frequencies
    #             when concave then model as sum of power laws.
    #             when convex then model as SSA and/or FF absorption to give low frequency turnover.
    #
    # freq_array: float array of frequencies (MHz) at which brightness data is available for current pixel.
    #
    # Tb_array: float array of kelvin brightness temperatures at the frequencies listed.


In [7]:
def pyGMOSS_Func_ComputeVariance(x, flag_param, freq_array, Tb_array):
    
    norm_param = x[0]
    alpha1_param = x[1]
    dalpha_param = x[2]
    nubrk_param = x[3]
    Tx_param = x[4]
    Te_param = x[5]
    nut_param = x[6]
    
    flag = flag_param
    freq_GHz = freq_array/1e3 # freq_array is in MHz
    
    GSPAN0 = 100.0  # defines a factor by which the range of integration is expanded beyond required boundary
                    # this is an initial guess; it's updated for each sky pixel to avoid discontinuities

    import numpy as np
    import scipy.constants
    PI = scipy.constants.pi
    q_e = scipy.constants.elementary_charge
    Bmag = 1e-9  # Tesla == 10 micro-Gauss
    sin_alph = 1.0
    m_e = scipy.constants.m_e
    cvel = scipy.constants.c
    scale_gam_nu = (3.0*q_e*Bmag*sin_alph)/(4.0*PI*m_e*cvel)  ## when multiplied by gamma^2 gives critical freq

    def modbessik2(u):  # Modified Bessel Fn of second kind 
                        # for non-negative real fractional order for real positive argument

        uu = max(0.01,u) # np.where ((u<0.01),0.01,u)
        bk = scipy.special.kve(5.0/3.0,1/uu,out=None)
        return (bk/(exp(1/uu)*uu*uu))

    def fofx1float(gama,nu,scale_gam_nu,C1,fmult):  

        nu_c = ( (gama*gama) * (scale_gam_nu/1e9) )
        x = (nu/nu_c)
        rint =  scipy.integrate.quad(modbessik2,0.0,(1.0/x))             
        p1 = -((2*C1) - 3.0)
        return fmult*rint[0]*(gama**p1)*(x)

    def integral_func1(p1,p2,a1,a2,a3,a4):
        return (scipy.integrate.quad(fofx1float,p1,p2,epsabs=1.0e-4/a4,epsrel=1e-08,\
                                     args=(a1,a2,a3,a4)))[0]

    integ1 = np.vectorize(integral_func1,excluded=['p1','p2','a2','a3','a4'])
    integ2 = np.vectorize(integral_func1,excluded=['p1','p2','a2','a3','a4'])

    Tx = 10.0**Tx_param
    Te = 10.0**Te_param
    nu_t = 10.0**nut_param
    alpha1 = 10.0**alpha1_param
    
    extn = np.exp(-1.0*((nu_t/freq_GHz)**2.1))
    
    if flag == 0:  ## the case where alpha2 steeper than alpha1

        fnorm = 10.0**norm_param
        alpha2 = alpha1 + 10.0**dalpha_param
        nu_break = 10.0**nubrk_param
        gama_break = np.sqrt((nu_break)/scale_gam_nu)
        xb = gama_break
        
        nu_min0 = freq_GHz[0]*1e9/GSPAN0
        nu_max0 = freq_GHz[-1]*1e9*GSPAN0
        xl0 = np.sqrt((nu_min0)/scale_gam_nu)
        xu0 = np.sqrt((nu_max0)/scale_gam_nu)
        
        if xl0 > xb or xu0 < xb:
            
            if xl0 > xb: 
                C1 = alpha2
            else: 
                C1 = alpha1
            fmult =  ((gama_break)**(2*C1-3))    
                
            xl = np.sqrt((freq_GHz*1e9)/(GSPAN0*scale_gam_nu))
            xu = np.sqrt((freq_GHz*1e9*GSPAN0)/scale_gam_nu)
            
            index_array = np.transpose([xl,xu,freq_GHz])
    
            cspectT = [(scipy.integrate.quad(fofx1float,xl[i],xu[i],\
                                      args=(freq_GHz[i],scale_gam_nu,C1,fmult)))[0] for i in freq_GHz_indices]
            cspect = fnorm*((freq_GHz**-2.0) * cspectT + Tx*(freq_GHz**-2.1))*extn + Te*(1.0-extn)
                                 
        else:
            
            xl1 = xl0
            xu1 = xb
            fmult1 = (gama_break)**(2*alpha1-3)
            xl2 = xb
            xu2 = xu0
            fmult2 = ((gama_break)**(2*alpha2-3))
                        
            cspectT1 = integ1(xl1,xu1,freq_GHz,scale_gam_nu,alpha1,fmult1)
            cspectT2 = integ1(xl2,xu2,freq_GHz,scale_gam_nu,alpha2,fmult2)
            
            cspect = fnorm * ((freq_GHz**-2.0)*np.add(cspectT1,cspectT2)+Tx*(freq_GHz**-2.1))*extn\
                                + Te*(1.0-extn)
            
    else:            ##   If data requires alpha2 flatter than alpha1, 
                     ##   then model as sum of power laws, i.e. steep and flat spectrum sources
            
        fnorm1 = norm_param
        fnorm2 = nubrk_param
        alpha2 = alpha1 - 10.0**dalpha_param
        
        cspect = fnorm1*( (freq_GHz**(-alpha1)) + fnorm2*(freq_GHz**(-alpha2)) + \
                    Tx*(freq_GHz**-2.1) )*extn + Te*(1.0 - extn)

    var_ModelData = np.sum( cspect-Tb_array )
    
    return var_ModelData